In [1]:
import sys
sys.path.append("..")

import collections
import itertools

import numpy as np

import vaalianalyysi as va

# Onko äänelläni väliä?

Tässä tarkastellaan äänien vaikuttavuutta eri kunnissa: monestako äänestäjästä tulos oli kiinni?

Lasken, että tulos oli kiinni $n$ äänestäjästä, jos paikkamäärien muuttamiseen olisi riittänyt, että $n$ äänestäjää tekee erilaisen päätöksen, eli:

* jos $n$ äänestäjää, jotka äänestivät puoluetta $A$, olisivatkin jättäneet äänestämättä;
* jos $n$ äänestäjää, jotka jättivät äänestämättä, olisivatkin äänestäneet puoluetta $A$;
* jos $n$ äänestäjää, jotka äänestivät puoluetta $A$, olisivatkin äänestäneet puoluetta $B$.

Toiset siirtymät ovat realistisempia kuin toiset (esimerkiksi siirtyminen Vihreistä Vasemmistoliittoon lienee todennäköisempää kuin Perussuomalaisista Vihreisiin), ja toiset paikkamäärien muutokset ovat vaikuttavampia kuin toiset (jos Vasemmistoliitto menettää paikan SDP:lle, on se vähemmän vaikuttavaa kuin jos se olisi menettänyt paikan Kokoomukselle). Mutta nämä ovat subjektiivisia asioita, enkä huomioi niitä tässä.

In [2]:
# Kuntavaalien 2021 tulokset
V = va.data.tulospalvelu.vaalit("KV-2021")

Seuraavat funktiot laskevat, montako paikkaa kunnissa on tarjolla, montako ääntä kukin puolue sai, ja miten paikat jakautuvat d'Hondtin mukaisesti.

In [3]:
# Montako valitaan tässä kunnassa?
def paikkoja(kunta_nro):
    df = V.tulokset_alueittain
    df = df[df.alueen_tyyppi == "K"]
    df = df[df.kunta_nro.astype(int) == int(kunta_nro)]
    assert len(df) == 1
    return df.iloc[0].paikkoja

# Montako ääntä kukin puolue sai (huomioiden vaaliliitot)?
def ääniä(kunta_nro):
    df = V.tulokset_ehdokkaittain
    df = df[df.alueen_tyyppi == "K"]
    df = df[df.kunta_nro.astype(int) == int(kunta_nro)]
    liitossa = collections.defaultdict(lambda: set())
    äänet = collections.defaultdict(lambda: 0)
    for _, row in df.iterrows():
        nro = row.vaaliliitto_nro
        nimi = row.puolue_lyhenne
        äänet[nro] += row.n
        liitossa[nro].add(nimi)
    nrot = sorted(list(äänet.keys()))
    return [äänet[n] for n in nrot], ["+".join(sorted(list(liitossa[n]))) for n in nrot]

# Montako paikkaa puolueet saavat, kun äänet ovat N ja valitaan k?
def dhondt(N, k):
    vertailuluvut = []
    for i, n in enumerate(N):
        for j in range(1, k + 1):
            vertailuluvut.append((n / j, i))

    äänet = [0] * len(N)
    for vl, i in sorted(vertailuluvut)[::-1][:k]:
        äänet[i] += 1
    return äänet

Tässä lasketaan, monestako äänestäjästä tulos on ollut kiinni.

In [4]:
# Eri tavat, jolla vaalitulos voisi muuttua dn äänellä.
def mutate(N0, k, dn):
    for i in range(len(N0)):
        N = N0[:]
        N[i] += dn
        yield N
    for i in range(len(N0)):
        N = N0[:]
        N[i] -= dn
        yield N
    for i in range(len(N0)):
        for j in range(len(N0)):
            if i == j:
                continue
            N = N0[:]
            N[i] += dn
            N[j] -= dn
            yield N

# Monestako äänestä on kiinni, että paikkamäärä muuttuu?
def kiinni(N0, k):
    d0 = dhondt(N0, k)
    for dn in itertools.count(1):
        for N in mutate(N0, k, dn):
            if dhondt(N, k) != d0:
                return dn, N

# Kuntakohtaisten tulosten laskenta

In [5]:
def report(nimi, dn, N, dN, Nt):
    if np.sum(dN) < np.sum(N):
        p = [p for a, b, p in zip(N, dN, Nt) if a != b][0]
        print(f"Kunnassa {nimi}: jos {p} äänestäjistä {dn} olisi jättänyt äänestämättä.")
    elif np.sum(dN) > np.sum(N):
        p = [p for a, b, p in zip(N, dN, Nt) if a != b][0]
        print(f"Kunnassa {nimi}:, jos nukkuvista {dn} olisi äänestänyt {p}.")
    else:
        pa = [p for a, b, p in zip(N, dN, Nt) if a < b][0]
        pb = [p for a, b, p in zip(N, dN, Nt) if a > b][0]
        print(f"Kunnassa {nimi}: jos {pb} äänestäjistä {dn} olisikin äänestänyt {pa}.")

tulos = collections.defaultdict(list)
kunnat = [(kunta.nimi_suomeksi, nro) for nro, kunta in V.kunnat.items()]
for nimi, nro in sorted(kunnat):
    N, Nt = ääniä(nro)
    k = paikkoja(nro)
    dn, dN = kiinni(N, k)
    report(f"{nimi} ({nro})", dn, N, dN, Nt)
    tulos[dn].append(nimi)

Kunnassa Akaa (020): jos KD+KESK+SIN äänestäjistä 16 olisikin äänestänyt VIHR.
Kunnassa Alajärvi (005): jos KOK äänestäjistä 9 olisikin äänestänyt VAS.
Kunnassa Alavieska (009): jos VAS äänestäjistä 5 olisi jättänyt äänestämättä.
Kunnassa Alavus (010): jos KD+PS äänestäjistä 10 olisikin äänestänyt SDP.
Kunnassa Asikkala (016): jos PS äänestäjistä 15 olisikin äänestänyt VIHR.
Kunnassa Askola (018): jos KESK+RKP äänestäjistä 4 olisikin äänestänyt VIHR.
Kunnassa Aura (019): jos KESK äänestäjistä 4 olisikin äänestänyt KOK.
Kunnassa Enonkoski (046): jos SDP+VAS äänestäjistä 10 olisikin äänestänyt PS.
Kunnassa Enontekiö (047): jos VIHR äänestäjistä 1 olisi jättänyt äänestämättä.
Kunnassa Espoo (049): jos KD äänestäjistä 39 olisikin äänestänyt KOK.
Kunnassa Eura (050): jos KOK äänestäjistä 9 olisikin äänestänyt VAS.
Kunnassa Eurajoki (051): jos KESK äänestäjistä 8 olisikin äänestänyt VAS.
Kunnassa Evijärvi (052): jos KD+KESK äänestäjistä 26 olisikin äänestänyt KOK.
Kunnassa Forssa (061): jos 

# Kunnat lukumäärittäin

In [6]:
#print("[", end='')
for k in sorted(tulos.keys()):
    t = tulos[k]
    print(f"Seurajavissa {len(t)} kunnassa tulos oli {k} äänestäjästä kiinni: ", end="")
    print(", ".join(sorted(t)))
    #print(f'["{k}", [', end='')
    #for k in sorted(t):
    #   print(f'"{k}", ', end='')
    #print(f']],', end='')
#print("]")

Seurajavissa 22 kunnassa tulos oli 1 äänestäjästä kiinni: Enontekiö, Hanko, Ilmajoki, Isojoki, Kihniö, Kolari, Konnevesi, Lapinlahti, Mänttä-Vilppula, Nousiainen, Pelkosenniemi, Pietarsaari, Pomarkku, Pornainen, Puolanka, Puumala, Ruokolahti, Rääkkylä, Sauvo, Soini, Taivassalo, Tervo
Seurajavissa 21 kunnassa tulos oli 2 äänestäjästä kiinni: Hyrynsalmi, Iitti, Kannonkoski, Kauhava, Kokemäki, Koski Tl, Lahti, Laihia, Lapinjärvi, Lestijärvi, Maalahti, Masku, Muhos, Paimio, Polvijärvi, Punkalaidun, Rautjärvi, Siikalatva, Simo, Somero, Säkylä
Seurajavissa 33 kunnassa tulos oli 3 äänestäjästä kiinni: Halsua, Harjavalta, Heinola, Hirvensalmi, Hämeenkyrö, Ii, Iisalmi, Ilomantsi, Joutsa, Juupajoki, Karkkila, Kuhmoinen, Lappajärvi, Laukaa, Merikarvia, Multia, Muonio, Nurmes, Rantasalmi, Reisjärvi, Ruovesi, Savitaipale, Siikainen, Siikajoki, Siilinjärvi, Sonkajärvi, Sotkamo, Teuva, Tohmajärvi, Ulvila, Utsjoki, Uusikaarlepyy, Äänekoski
Seurajavissa 16 kunnassa tulos oli 4 äänestäjästä kiinni: Asko